In [ ]:
import tensorflow
tensorflow.keras.__version__

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [ ]:
import pandas as pd
df = pd.read_csv("HR_Mockdata.csv")
df = df.drop(columns=["sales \nfunction", "hire_date", 'move_date', 'end_date',
                    'country_home', 'country_host', 'business Size_host', 
                      'economic Perspective_host', 'left'])
# Drop null columns/rows
df = df.dropna(axis='columns', how='all')
df = df.dropna()
df.tail()

In [ ]:
list(df.columns.values)

In [4]:
# select column names that need to be converted to dummy variables
to_dummify = [
 'salary',
 'leader_host',
 'culture_home',
 'culture_host',
 'function_host',
 'fired_quit',
 ]

In [5]:
for dummy in to_dummify:
    df = pd.concat([df.drop(dummy, axis=1), pd.get_dummies(df[dummy], 
                prefix=dummy)], axis=1)
df.head()

In [6]:
X = df.drop("buzz_bust", axis=1)
y = df["buzz_bust"]
print(X.shape, y.shape)
X.head(1)

(14999, 38) (14999,)


,#,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,salary_high,salary_low,...,function_host_human resources,function_host_legal,function_host_market access,function_host_marketing,function_host_medical,function_host_operations,function_host_regulatory affairs,fired_quit_fired,fired_quit_no,fired_quit_quit
0,45,0.09,0.89,6,276,4,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    random_state=1, stratify=y)

In [9]:
from sklearn.preprocessing import MinMaxScaler
# Create a StandardScater model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

C:\Users\h4wth\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [10]:
# Transform the training and testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)